In [13]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import pandas as pd
import numpy as np

In [12]:
n,m=943,1682
cols = ['user_id', 'item_id', 'rating', 'ts']
base = pd.read_csv('../../ml-100k/u1.base',sep='\t',names=cols)
test = pd.read_csv('../../ml-100k/u1.test',sep='\t',names=cols)

In [16]:
# 创建uesr-item矩阵
train_matrix = np.zeros((n, m))
test_matrix = np.zeros((n, m))
y_ui = np.zeros((n, m))
for i in base.itertuples():
    train_matrix[i[1] - 1, i[2] - 1] = i[3]

for i in test.itertuples():
    test_matrix[i[1] - 1, i[2] - 1] = i[3]

# 创建y矩阵
for i in range(n):
    for j in range(m):
        if train_matrix[i][j] != 0:
            y_ui[i][j] = 1

In [17]:
# 统计指标的计算

In [19]:
global_average_rating = train_matrix.sum()/y_ui.sum()

In [36]:
avg_rating_user = np.array([0.0]*n)
avg_rating_item = np.array([0.0]*m)

In [37]:
for i in range(n):
    if y_ui[i].sum()==0:
        avg_rating_user[i] = global_average_rating
    else:
        avg_rating_user[i] = train_matrix[i].sum()/y_ui[i].sum()

In [39]:
for i in range(m):
    if y_ui[:,i].sum()==0:
        avg_rating_item[i] = global_average_rating
    else:
        avg_rating_item[i] = train_matrix[:,i].sum()/y_ui[:,i].sum()

In [52]:
bias_of_user = np.array([0.0]*n)
bias_of_item = np.array([0.0]*m)

In [53]:
for i in range(n):
    if y_ui[i].sum()==0:
        bias_of_user[i] = 0.0
    else:
        bias_of_user[i] = sum((train_matrix[i]-avg_rating_item)*y_ui[i])/y_ui[i].sum()

In [54]:
for i in range(m):
    if y_ui[:,i].sum()==0:
        bias_of_item[i] =  0.0
    else:
        bias_of_item[i] = sum((train_matrix[:,i]-avg_rating_user)*y_ui[:,i])/y_ui[:,i].sum() 

In [55]:
# 评分预测公式

In [76]:
user_avg_p = np.zeros((n,m))+avg_rating_user[:,np.newaxis]
item_avg_p = np.array([avg_rating_item]*943)
mean_user_avg_item_avg = np.array([avg_rating_user/2]).T+np.array([avg_rating_item/2])
user_bias_item_avg = np.array([bias_of_user]).T+np.array([avg_rating_item])
user_avg_item_bias = np.array([avg_rating_user]).T+np.array([bias_of_item])
global_avg_user_bias_item_bias = global_average_rating+np.array([bias_of_user]).T+np.array([bias_of_item])

In [80]:
# 评估函数
def eva(prediction, truth):
    prediction = prediction[truth.nonzero()].flatten()
    truth = truth[truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, truth)),mean_absolute_error(prediction, truth)

In [89]:
print('user average RMSE、MAE: ' + str(eva(user_avg_p, test_matrix)))
print('item average RMSE、MAE: ' + str(eva(item_avg_p, test_matrix)))
print('mean of user average and item average RMSE、MAE: ' + str(eva(mean_user_avg_item_avg, test_matrix)))
print('user bias and item average RMSE、MAE: ' + str(eva(user_bias_item_avg, test_matrix)))
print('user average and item bias RMSE、MAE: ' + str(eva(user_avg_item_bias, test_matrix)))
print('global average, user bias and item bias RMSE、MAE: ' + str(eva(global_avg_user_bias_item_bias, test_matrix)))

user average RMSE、MAE: (1.0629951276561334, 0.8501912740150434)
item average RMSE、MAE: (1.0334113714152895, 0.8275684032890005)
mean of user average and item average RMSE、MAE: (0.9985125029305972, 0.8084812824773311)
user bias and item average RMSE、MAE: (0.9602429647790681, 0.7573844292873362)
user average and item bias RMSE、MAE: (0.9757862482949855, 0.7696373147206427)
global average, user bias and item bias RMSE、MAE: (0.962331641550567, 0.7612786028606267)
